In [7]:
import pandas as pd

cart = pd.read_csv('cart.csv')
checkout = pd.read_csv('checkout.csv')
purchase = pd.read_csv('purchase.csv')
visits = pd.read_csv('visits.csv')

print(cart.head())
print(checkout.head())
print(purchase.head())
print(visits.head())

#DataFrames should be printed due to "no common columns" and "no variables defined" errors.

                                user_id            cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168  2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973  2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663  2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3  2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974  2017-07-26 15:48:00
                                user_id        checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d  2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e  2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d  2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a  2017-04-20 16:15:00
4  3ccdaf69-2d30-40de-b083-51372881aedd  2017-01-08 20:52:00
                                user_id        purchase_time
0  4b44ace4-2721-47a0-b24b-15fbfa2abf85  2017-05-11 04:25:00
1  02e684ae-a448-408f-a9ff-dcb4a5c99aac  2017-09-05 08:45:00
2  4b4bc391-749e-4b90-ab8f-4f6e3c84d6dc  2017-11-20 20:49:00
3  a5dbb25f-3c36-4103-90

### Left-merging visits and cart.

In [12]:
cart_visits = pd.merge(
    visits,
    cart,
    how = 'left')

cart_visits.head()

,user_id,visit_time,cart_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaN
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaN
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaN


In [11]:
cart_visits_len = len(cart_visits)
cart_visits_len

2000

### Counting null timestamps.

In [15]:
cart_visits_null = len(cart_visits[cart_visits.cart_time.isnull()])
cart_visits_null

1652

###  Percent of users who visited website ended up not placing a t-shirt in their cart.

In [17]:
print(float(cart_visits_null)/cart_visits_len)

0.826


###  Left merge for cart and checkout and counting null values. The percentage of users who put items in their cart, but did not proceed to checkout.

In [20]:
cart_checkout = pd.merge(
    cart,
    checkout,
    how = 'left')

cart_checkout_len = len(cart_checkout)
cart_checkout_null = len(cart_checkout[cart_checkout.checkout_time.isnull()])

print(float(cart_checkout_null / cart_checkout_len))

0.25311203319502074


### The percentage of users who proceeded to checkout, but did not purchased item.

In [24]:
checkout_purchase = pd.merge(
    checkout,
    purchase,
    how = 'left')

checkout_purchase_len = len(checkout_purchase)
checkout_purchase_null = len(checkout_purchase[checkout_purchase.purchase_time.isnull()])

print(float(checkout_purchase_null / checkout_purchase_len))

0.1688963210702341


### Merge of all four steps of the funnel, in order, using a series of left merges.

In [88]:
all_data = visits\
    .merge(cart, how = 'left')\
    .merge(checkout, how = 'left')\
    .merge(purchase, how = 'left')
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaN,NaN,NaN
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaN,NaN
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaN,NaN,NaN
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaN,NaN,NaN


### Calculating average visit-to-purchase time.

In [106]:
#I have noticed that there is 'unsupported operant type' error caused by the fact that my dataframe consists of objetcs not datetime64.
#So, I turned colums of interest into datetime64 to calculate mean.

all_data['visit_time'] = pd.to_datetime(all_data['visit_time'])
all_data['purchase_time'] = pd.to_datetime(all_data['purchase_time'])

all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2372 entries, 0 to 2371
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   user_id           2372 non-null   object         
 1   visit_time        2372 non-null   datetime64[ns] 
 2   cart_time         720 non-null    object         
 3   checkout_time     598 non-null    object         
 4   purchase_time     497 non-null    datetime64[ns] 
 5   time_to_purchase  497 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), object(3), timedelta64[ns](1)
memory usage: 129.7+ KB


In [108]:
all_data['time_to_purchase'] = \
  all_data.purchase_time - \
  all_data.visit_time

print(all_data.time_to_purchase.head())
print(all_data.time_to_purchase.mean())

0               NaT
1   0 days 00:44:00
2               NaT
3               NaT
4               NaT
Name: time_to_purchase, dtype: timedelta64[ns]
0 days 00:43:53.360160965
